# Running Faces pipeline

In [1]:
import sys
import subprocess
import pkg_resources
import IPython

required = {'mlrun'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed
previously_installed = required.intersection(installed)

if missing:
    print(f'Installing {",".join(missing)}')
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)
    print('Restarting kernel')
    IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel
if previously_installed:
    print(f'Already installed: {",".join(previously_installed)}')

Already installed: mlrun


### create the mlrun project 

In [2]:
from os import path, getenv
from mlrun import new_project, mlconf

#project_name = '-'.join(filter(None, ['getting-started-iris', getenv('V3IO_USERNAME', None)]))
project_name = "faces"
project_path = path.abspath('./')
project = new_project(project_name, project_path)
project.save()
print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/mlrun/demos/faces/notebooks
Project name: faces


In [3]:
out = mlconf.artifact_path or path.abspath('./data')
# {{run.uid}} will be substituted with the run id, so output will be written to different directoried per run
artifact_path = path.join(out, '{{run.uid}}')
%env PYTHONPATH=./

env: PYTHONPATH=./


In [4]:
from mlrun import mount_v3io, code_to_function
encode_images_func = code_to_function('encode_images', kind='job', filename='functions/encode_images.py')
encode_images_func.spec.build.commands = ['pip install face_recognition imutils opencv-python-headless v3io_frames==0.7.37']
encode_images_func.spec.build.base_image = 'mlrun/ml-models'
encode_images_func.deploy()

> 2020-11-11 13:42:46,997 [info] starting remote build, image: .mlrun/func-default-encode-images-latest
INFO[0000] Resolved base name mlrun/ml-models:0.5.4 to mlrun/ml-models:0.5.4 
INFO[0000] Resolved base name mlrun/ml-models:0.5.4 to mlrun/ml-models:0.5.4 
INFO[0000] Retrieving image manifest mlrun/ml-models:0.5.4 
INFO[0002] Retrieving image manifest mlrun/ml-models:0.5.4 
INFO[0004] Built cross stage deps: map[]                
INFO[0004] Retrieving image manifest mlrun/ml-models:0.5.4 
INFO[0005] Retrieving image manifest mlrun/ml-models:0.5.4 
INFO[0006] Unpacking rootfs as cmd RUN pip install face_recognition imutils opencv-python-headless v3io_frames==0.7.37 requires it. 
INFO[0149] Taking snapshot of full filesystem...        
INFO[0179] Resolving paths                              
INFO[0204] RUN pip install face_recognition imutils opencv-python-headless v3io_frames==0.7.37 
INFO[0204] cmd: /bin/sh                                 
INFO[0204] args: [-c pip install face_recog

True

In [5]:
from mlrun import mount_v3io, code_to_function
train_func = code_to_function('train', kind='job', filename='functions/train.py')
train_func.spec.build.commands = ['pip install v3io_frames==0.7.37']
train_func.spec.build.base_image = 'mlrun/ml-models'
train_func.deploy()

> 2020-11-11 13:49:47,715 [info] starting remote build, image: .mlrun/func-default-train-latest
INFO[0000] Resolved base name mlrun/ml-models:0.5.4 to mlrun/ml-models:0.5.4 
INFO[0000] Resolved base name mlrun/ml-models:0.5.4 to mlrun/ml-models:0.5.4 
INFO[0000] Retrieving image manifest mlrun/ml-models:0.5.4 
INFO[0002] Retrieving image manifest mlrun/ml-models:0.5.4 
INFO[0003] Built cross stage deps: map[]                
INFO[0003] Retrieving image manifest mlrun/ml-models:0.5.4 
INFO[0004] Retrieving image manifest mlrun/ml-models:0.5.4 
INFO[0006] Unpacking rootfs as cmd RUN pip install v3io_frames==0.7.37 requires it. 
INFO[0152] Taking snapshot of full filesystem...        


2020-11-11 13:52:50,110 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPConnectionPool(host='mlrun-api', port=8080): Read timed out. (read timeout=20)")': /api/build/status?name=train&project=default&tag=&logs=yes&offset=676&last_log_timestamp=0&verbose=no


INFO[0180] Resolving paths                              
INFO[0205] RUN pip install v3io_frames==0.7.37          
INFO[0205] cmd: /bin/sh                                 
INFO[0205] args: [-c pip install v3io_frames==0.7.37]   
INFO[0211] Taking snapshot of full filesystem...        
INFO[0212] Resolving paths                              
INFO[0232] RUN pip install mlrun                        
INFO[0232] cmd: /bin/sh                                 
INFO[0232] args: [-c pip install mlrun]                 
INFO[0235] Taking snapshot of full filesystem...        
INFO[0235] Resolving paths                              


True

### set the project functions

In [7]:
from mlrun import mount_v3io, code_to_function
#project.set_function("functions/clean_parquez.py", 'clean', kind='job', image='aviaigz/parquez')

ARTIFACTS_PATH ='/User/faces/artifacts/'

project.set_function(encode_images_func,name = 'encode_images')
project.set_function(train_func,name = 'train')
project.func('encode_images').apply(mount_v3io())
project.func('train').apply(mount_v3io())
project.func('encode_images').set_env('PYTHONPATH', project_path)
project.func('train').set_env('PYTHONPATH', project_path)
project.func('encode_images').spec.artifact_path = ARTIFACTS_PATH
project.func('train').spec.artifact_path = ARTIFACTS_PATH



<a id="gs-step-create-n-run-ml-pipeline"></a>
## Create and Run a Fully Automated ML Pipeline

You're now ready to create a full ML pipeline.
This is done by using [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/), which is integrated into the Iguazio Data Science Platform.
Kubeflow Pipelines is an open-source framework for building and deploying portable, scalable machine-learning workflows based on Docker containers.
MLRun leverages this framework to take your existing code and deploy it as steps in the pipeline.

In [8]:
%%writefile {path.join(project_path, 'workflow.py')}

from kfp import dsl
from mlrun import mount_v3io
from os import getenv, path

DATA_PATH ='/User/faces/dataset/'
ARTIFACTS_PATH ='/User/faces/artifacts/'
MODELS_PATH = '/User/mlrun/demos/faces/notebooks/functions/models.py'
FRAMES_URL = 'framesd:8081'
V3IO_ACCESS_KEY = getenv('V3IO_ACCESS_KEY')
USER_NAME = getenv('V3IO_USERNAME')
ENCODINGS_PATH = '/'.join([USER_NAME,'faces','encodings']) 
WEB_API = "http://v3io-webapi:8081"


funcs = {}
project_path = path.abspath('./')
faces_params = {'data_path' : DATA_PATH,
                'artifacts_path': ARTIFACTS_PATH,
                'models_path': MODELS_PATH,
                'frames_url': FRAMES_URL,
                'token' : V3IO_ACCESS_KEY, 
                'encodings_path': ENCODINGS_PATH }

# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    project_path = path.abspath('./')
    for f in functions.values():
        f.apply(mount_v3io())
        f.set_env('PYTHONPATH', project_path)
        f.spec.artifact_path = ARTIFACTS_PATH
        
        
        
# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name = "faces-pipeline",
    description = "faces demo pipeline"
)
def kfpipeline():

    # encode images
    encode = funcs['encode_images'].as_step(
        name="encode_images",
        params=faces_params,
        outputs=['encode']
    )
    
    # train the model based on the images
    train = funcs['train'].as_step(
        name="train",
        params = faces_params,
        inputs={'table': encode.outputs},                       
        outputs=['training']
    )

Writing /User/mlrun/demos/faces/notebooks/workflow.py


<a id="gs-register-workflow"></a>
#### Register the Workflow

Use the `set_workflow` MLRun project method to register your workflow with MLRun.
The following code sets the `name` parameter to the selected workflow name ("main") and the `code` parameter to the name of the workflow file that is found in your project directory (**workflow.py**).

In [9]:
# Register the workflow file as "main"
project.set_workflow('main', 'workflow.py')

In [10]:
project.save()

In [11]:
run_id = project.run(
    'main',
    arguments={}, 
    
    artifact_path=path.abspath(path.join('pipeline','{{workflow.uid}}'),
    
                              )
    ,dirty=True)

> 2020-11-11 13:55:45,238 [info] using in-cluster config.


> 2020-11-11 13:55:45,691 [info] Pipeline run id=e6da3203-49ec-409f-8b65-0fbe685f21fc, check UI or DB for progress
